### Set up

#### 1. Set  up  accounts and role

In [1]:
## !pip install sagemaker --upgrade

In [2]:
import sagemaker
import boto3
import copy

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


#### 2. Setup image and instance type

In [3]:
pytorch_custom_image_name="ppi-extractor:gpu-1.0.0-201910130520"
instance_type = "ml.p3.2xlarge" 

In [4]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

In [5]:
commit_id= '47734237f065dd5f5f694e7084dff5e2a48de87d'

In [6]:
bucket = "aegovan-data"

In [7]:



trainfile = "s3://{}/processed_dataset/train_classification.json".format(bucket)
testfile= "s3://{}/processed_dataset/test_classification.json".format(bucket)
valfile="s3://{}/processed_dataset/val_classification.json".format(bucket)





binary_trainfile = "s3://{}/processed_dataset/train_binary_classification.json".format(bucket)
binary_testfile= "s3://{}/processed_dataset/test_binary_classification.json".format(bucket)
binary_valfile="s3://{}/processed_dataset/val_binary_classification.json".format(bucket)



pretrained_bert="s3://{}/embeddings/bert/".format(bucket)


s3_output_path= "s3://{}/results/".format(bucket)
s3_code_path= "s3://{}/aimed_bert_code".format(bucket)

### Start training

In [8]:
max_runs = 1

In [9]:
BertNetworkFactoryhyperparameters = {
    "dataset":"InteractionDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile.split("/")[-1],
    "valfile":valfile.split("/")[-1],
    "testfile":testfile.split("/")[-1],
    "batchsize": "8",
    "accumulation_steps": 8,
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    "git_commit_id" : commit_id,
    "earlystoppingpatience":20,


}

simple_inputs = {
    "train" : trainfile,
    "val" :valfile,
    "test" : testfile,
    "PRETRAINED_BIOBERT" : pretrained_bert
}

inputs_simple_hp ={
    "inputs":simple_inputs,
    "hp":BertNetworkFactoryhyperparameters
}

In [10]:
mini_BertNetworkFactoryhyperparameters = copy.deepcopy(BertNetworkFactoryhyperparameters)

mini_BertNetworkFactoryhyperparameters["network"] ="RelationExtractorMiniBioBertFactory"
mini_BertNetworkFactoryhyperparameters["num_layers"] = "5"

inputs_mini_simple_hp ={
    "inputs":simple_inputs,
    "hp":mini_BertNetworkFactoryhyperparameters
}

In [11]:
binary_BertNetworkFactoryhyperparameters = copy.deepcopy(BertNetworkFactoryhyperparameters)

binary_BertNetworkFactoryhyperparameters["trainfile"] = binary_trainfile.split("/")[-1]
binary_BertNetworkFactoryhyperparameters["valfile"] = binary_valfile.split("/")[-1]
binary_BertNetworkFactoryhyperparameters["testfile"] = binary_testfile.split("/")[-1]
binary_BertNetworkFactoryhyperparameters["dataset"] = "InteractionBinaryDatasetFactory"

binary_inputs = {
    "train" : binary_trainfile,
    "val" :binary_valfile,
    "test" : binary_testfile,
    "PRETRAINED_BIOBERT" : pretrained_bert
}

inputs_binary_hp ={
    "inputs":binary_inputs,
    "hp":binary_BertNetworkFactoryhyperparameters
}


In [12]:
mini_binary_BertNetworkFactoryhyperparameters = copy.deepcopy(binary_BertNetworkFactoryhyperparameters)

mini_binary_BertNetworkFactoryhyperparameters["network"] ="RelationExtractorMiniBioBertFactory"
mini_binary_BertNetworkFactoryhyperparameters["num_layers"] = "5"

inputs_mini_binary_hp ={
    "inputs":binary_inputs,
    "hp":mini_binary_BertNetworkFactoryhyperparameters
}



In [13]:
all_inputs_hp = {
    "simple" :inputs_simple_hp,
    "minisimple" : inputs_mini_simple_hp,
    "binary" : inputs_binary_hp,
    "minibinary" : inputs_mini_binary_hp
    
}

In [14]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainFScore",
                     "Regex": "###score: train_fscore### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationFScore",
                     "Regex": "###score: val_fscore### (\d*[.]?\d*)"}
                    ]

In [15]:
!git log -1 | head -1
!git log -1 | head -5 | tail -1

commit 47734237f065dd5f5f694e7084dff5e2a48de87d
    Fix class name


In [16]:
job_type = "minibinary"
base_name = "i-class-bert-{}".format(job_type)
hyperparameters = all_inputs_hp[job_type]["hp"]  
inputs = all_inputs_hp[job_type]["inputs"]


In [17]:
git_config = {'repo': 'https://github.com/elangovana/PPI-typed-relation-extractor.git',
              'branch': 'master',
             'commit': hyperparameters['git_commit_id']
             }

In [18]:
hyperparameters

{'dataset': 'InteractionBinaryDatasetFactory',
 'network': 'RelationExtractorMiniBioBertFactory',
 'trainfile': 'train_binary_classification.json',
 'valfile': 'val_binary_classification.json',
 'testfile': 'test_binary_classification.json',
 'batchsize': '8',
 'accumulation_steps': 8,
 'epochs': '1000',
 'log-level': 'INFO',
 'learningrate': 1e-05,
 'git_commit_id': '47734237f065dd5f5f694e7084dff5e2a48de87d',
 'earlystoppingpatience': 20,
 'num_layers': '5'}

In [19]:
inputs

{'train': 's3://aegovan-data/processed_dataset/train_binary_classification.json',
 'val': 's3://aegovan-data/processed_dataset/val_binary_classification.json',
 'test': 's3://aegovan-data/processed_dataset/test_binary_classification.json',
 'PRETRAINED_BIOBERT': 's3://aegovan-data/embeddings/bert/'}

In [20]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     #entry_point='main_train_k_fold.py',
    entry_point='main_train_bert.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets', 'source/preprocessor', 'source/modelnetworks','source/trainpipelinesbuilders', 'source/metrics'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                    git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    code_location=s3_code_path,
                    base_job_name =base_name)

In [ ]:
estimator.fit(inputs, wait=False)

# Results